Pre processing meteorological data

# Intro

## Libraries

In [1]:
import os
import glob
from datetime import datetime

import pandas as pd
import numpy as np
from decimal import Decimal
import collections
import time

from sklearn import datasets, linear_model
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

import statsmodels.formula.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from brokenaxes import brokenaxes

from scipy.stats import ttest_ind, ttest_ind_from_stats
from scipy.special import stdtr

from functools import reduce

# from co2mpas.datasync import _get_interp_method

## Input files

In [2]:
# ESTACIONES 1

path1 = r'C:/CicloHidrico/'         # directorio de las estaciones
os.chdir(path1)

estID1 = pd.read_csv("estaciones.csv", sep=';', engine='python', index_col=1)                   # estaciones con ID number
estRang1 = pd.read_excel("EstacionesTermoPluvio.xlsx", header=1, sep=';', usecols=range(1, 10)) # estaciones con rango de fecha

# print(os.getcwd())
# # Display all of the files found in your current working directory
# print(os.listdir(os.getcwd()))

In [3]:
# ESTACIONES 2 (nuevas de CU, AB, TE)

path2 = r'C:/CicloHidrico/AemetBernat'      # directorio de los datos
os.chdir(path2)

estUTM2 = pd.read_excel("Existencias_estaciones_climatologicas_CuAbTe.xls", header=0, sep=';', index_col=0) # 636 con UTM

path3 = r'C:/CicloHidrico/DB_Paco'      # directorio de los datos
os.chdir(path3)

estnew2 = pd.read_csv("estacions_AEMET_Bernat.csv", sep=';', engine='python', index_col=0)    # estaciones con ID number

In [4]:
# DATOS PRECIPITACIÓN 
precipitacion = pd.read_csv("precipitacion_AEMET.csv", sep=';',                # precipitacion total diaria DB
                  names=["CodigoEst", "Fecha", "Precipitacion", "ValidPP"],
#                   index_col=0
                           )   

C:\Users\ceam\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
precipitacion.drop(precipitacion[precipitacion.CodigoEst=='Codig'].index, inplace=True)

In [6]:
# DATOS TEMPERATURAS

t_max = pd.read_csv("temperatura_max(1).csv", sep=';',                         # temperaturas màximas diarias
                    names=["CodigoEst", "Fecha", "Tmaxima", "ValidTmax"],
#                     index_col=0
                   )      

t_min = pd.read_csv("temperatura_min(1).csv", sep=';',                         # temperaturas mínimas diarias 
                    names=["CodigoEst", "Fecha", "Tminima", "ValidTmin"],
#                     index_col=0
                   )                                               

In [69]:
t_max.head()

,CodigoEst,Fecha,Tmaxima,ValidTmax
0,7031,1958-01-01,14.5,S
1,7031,1958-01-02,16.0,S
2,7031,1958-01-03,15.0,S
3,7031,1958-01-04,13.5,S
4,7031,1958-01-05,18.0,S


# Pre processing

## Datos de las estaciones

### Formato del Código

In [7]:
# Todos los códigos son strings

estID1.index = estID1.index.astype('str')
estUTM2.index = estUTM2.index.astype('str')
estnew2.index = estnew2.index.astype('str')
precipitacion.CodigoEst = precipitacion.CodigoEst.astype('str')
t_max.CodigoEst = t_max.CodigoEst.astype('str')
t_min.CodigoEst = t_min.CodigoEst.astype('str')

### Unir estaciones 1

In [8]:
estID1 = estID1.reset_index().drop(columns=['Unnamed: 0']) # CódigoEst fuera del índice

In [10]:
estRang1.head()

,ESTACIÓN,Longitud,Latitud,Altura,Provincia,Cobertura Temperatura,Cobertura Precipitación,Temp,Pluv
0,A CORUNYA (ESTACION COMPLETA),-8.42,43.37,58,AC,<->,1930-10-01<->2010-10-31,NO,SI
1,ABABUJ,-1.28,40.56,1368,TE,<->,<->,NO,NO
2,"ABANILLA, CHS",-1.03,38.20,222,MU,1958-01-01<->2010-10-31,1958-01-01<->2010-10-31,SI,SI
3,ABARAN (SIERRA DE LA PILA),-1.32,38.25,300,MU,2007-10-01<->2010-10-31,1961-04-01<->2010-10-31,SI,SI
4,ABARAN (SIERRA DEL ORO),-1.42,38.18,400,MU,2007-10-01<->2010-10-31,1957-04-01<->2010-10-31,SI,SI


In [11]:
estRang1 = estRang1.rename(columns={'Longitud': 'Long', 'Latitud': 'Lat', 'Provincia':'Prov'}) # evita columnas con mismo nombre
estaciones1 = pd.merge(estID1, estRang1, left_on='Estacion', right_on='ESTACIÓN', how='inner')                                                            # concatena DFs

estaciones1.head()

,CodigoEst,Estacion,Tipo,Longitud,Latitud,Altitud,Xutm,Yutm,Provincia,Umbral_20_40_km_costa,ESTACIÓN,Long,Lat,Altura,Prov,Cobertura Temperatura,Cobertura Precipitación,Temp,Pluv
0,1104,MIRONES,INMman,#¡CAMPO!,43.52,200,442832,4793841,CN,2,MIRONES,-3.71,43.52,200,CN,<->,1968-01-01<->2010-10-31,NO,SI
1,1212E,RANON (AEROPUERTO DE ASTURIAS),INMman,#¡CAMPO!,43.79,127,255062,4828087,AS,1,RANON (AEROPUERTO DE ASTURIAS),-6.04,43.79,127,AS,<->,1968-08-01<->2010-10-31,NO,SI
2,1259,GENESTOSO,INMman,#¡CAMPO!,43.28,1180,223429,4773596,AS,3,GENESTOSO,-6.40,43.28,1180,AS,<->,1961-01-01<->2010-10-31,NO,SI
3,1387,A CORUNYA (ESTACION COMPLETA),INMman,#¡CAMPO!,43.37,58,59149,4791283,AC,1,A CORUNYA (ESTACION COMPLETA),-8.42,43.37,58,AC,<->,1930-10-01<->2010-10-31,NO,SI
4,1428,SANTIAGO COMPOSTELA (LABACOLLA,INMman,#¡CAMPO!,42.90,364,55186,4739467,AC,2,SANTIAGO COMPOSTELA (LABACOLLA,-8.43,42.90,364,AC,<->,1943-11-01<->2010-10-31,NO,SI


In [12]:
estaciones1 = pd.merge(estID1, estRang1, left_on='Estacion', right_on='ESTACIÓN', how='inner')                 # concatena DFs

In [13]:
estaciones1 = estaciones1.drop_duplicates(subset='CodigoEst').reset_index(drop=True) 

In [14]:
estaciones1.tail()

,CodigoEst,Estacion,Tipo,Longitud,Latitud,Altitud,Xutm,Yutm,Provincia,Umbral_20_40_km_costa,ESTACIÓN,Long,Lat,Altura,Prov,Cobertura Temperatura,Cobertura Precipitación,Temp,Pluv
897,IVI47,"CREVILLENT, IVIA",IVIAaut,#¡CAMPO!,38.22,84,692560,4234063,A,1,"CREVILLENT, IVIA",-0.80,38.22,84,A,2001-07-16<->2004-03-11,2001-07-16<->2004-03-11,SI,SI
898,IVI48,"ALMORADI, IVIA",IVIAaut,#¡CAMPO!,38.11,9,695790,4221185,A,1,"ALMORADI, IVIA",-0.77,38.11,9,A,2001-07-16<->2004-03-11,2001-07-16<->2004-03-11,SI,SI
899,IVI49,"CATRAL, IVIA",IVIAaut,#¡CAMPO!,38.15,9,689858,4224746,A,1,"CATRAL, IVIA",-0.83,38.15,9,A,2001-06-28<->2004-03-11,2001-06-28<->2004-03-11,SI,SI
900,IVI50,"ORIHUELA, LA MURADA, IVIA",IVIAaut,#¡CAMPO!,38.18,86,678092,4228180,A,2,"ORIHUELA, LA MURADA, IVIA",-0.97,38.18,86,A,2001-06-28<->2004-03-11,2001-06-28<->2004-03-11,SI,SI
901,IVI51,"PILAR DE LA HORADADA, IVIA",IVIAaut,#¡CAMPO!,37.87,10,694990,4193406,A,1,"PILAR DE LA HORADADA, IVIA",-0.78,37.87,10,A,2001-06-28<->2004-03-11,2001-06-28<->2004-03-11,SI,SI


### Unir estaciones 2

In [15]:
estaciones2 = pd.merge(estnew2, estUTM2, left_index=True, right_index=True)

In [16]:
estaciones2 = estaciones2.reset_index().rename(columns={'index':'CodigoEst'})

In [17]:
estaciones2.head()

,CodigoEst,Estacion,Tipo,Longitud_x,Latitud_x,Altitud,Xutm,Yutm,Provincia_x,Umbral_20_40_km_costa,...,Provincia_y,Fec.Ini,Fec.Fin,Meses,AÃ±os C.,AÃ±os I.,Serie,Sig.Serie,UTM X,UTM Y
0,3010,RODENAS,INMman,-15.169.444.444.444.400,4.063.916.666.666.660,1370,0,0,TE,NaN,...,TERUEL,1968,2019,554,36,15,1992 2001,1974 1983,625401,4499756
1,3042,VEGA DEL CODORNO,INMman,-1.932.222.222.222.220,4.043.222.222.222.220,1345,0,0,CU,NaN,...,CUENCA,1956,2019,739,53,10,1956 1994,1999 2004,590563,4476275
2,3051,POYATOS (TEJADILLOS) C.F.,INMman,-1.986.388.888.888.880,4.039.472.222.222.220,1150,0,0,CU,NaN,...,CUENCA,1952,2005,612,41,13,1952 1974,1981 1987,586016,4472058
3,3058,TORRECILLA,INMman,-2.207.222.222.222.220,4.029.194.444.444.440,969,0,0,CU,NaN,...,CUENCA,1955,2019,708,51,11,1957 1979,2008 2018,567376,4460458
4,3059,ALBALATE DE LAS NOGUERAS,INMman,-22.780.555.555.555.500,4.036.555.555.555.550,855,0,0,CU,NaN,...,CUENCA,1940,2019,845,57,19,1962 1982,1998 2011,561288,4468577


### Unir estaciones 1 y 2

In [18]:
estaciones = pd.merge(estaciones1[['Estacion','CodigoEst','Tipo','Xutm','Yutm','Altura','Provincia','Umbral_20_40_km_costa']],
                      estaciones2[['Estacion','CodigoEst','Tipo','UTM X','UTM Y','Altitud','Provincia_x']],
                      on='CodigoEst', how='outer')  # se queda con todas las estaciones y rellana con NaN lo que no tiene

In [19]:
values = {'Estacion_x': estaciones.Estacion_y, 'Estacion_y':estaciones.Estacion_x,
          'Tipo_x':estaciones.Tipo_y,'Tipo_y':estaciones.Tipo_x,
          'Xutm':estaciones['UTM X'], 'Yutm':estaciones['UTM Y'],'UTM X':estaciones['Xutm'], 'UTM Y':estaciones['Yutm'],
          'Altura':estaciones.Altitud,'Altitud':estaciones.Altura,
          'Provincia':estaciones.Provincia_x, 'Provincia_x':estaciones.Provincia,
          'Umbral_20_40_km_costa':3 # NaN umbral corresponden a CU, AB y TE --> 3
         }
estaciones = estaciones.fillna(value=values) # rellena los NaN en modo 'espejo'
estaciones = estaciones.drop(columns=['Estacion_y','Tipo_y','UTM X','UTM Y','Altitud','Provincia_x']) # elimina medio espejo
estaciones.columns = ['Estacion','CodigoEst','Tipo','Xutm','Yutm','Altura','Provincia','Umbral_20_40_km_costa']


In [20]:
estaciones.tail(3)

,Estacion,CodigoEst,Tipo,Xutm,Yutm,Altura,Provincia,Umbral_20_40_km_costa
940,ALIAGA (COMARCAL),9556U,INMman,693955.0,4504500.0,1115.0,TE,3.0
941,ALIAGA-SANTA BÁRBARA,9557B,INMman,692954.0,4505280.0,1105.0,TE,3.0
942,VALDERROBRES (COMARCAL),9935A,INMman,765138.0,4529462.0,482.0,TE,3.0


### Datos duplicados

In [166]:
# """
# Estaciones con el nombre repetido, con el Código repetido o con las coordenadas geográficas repetidas
# """
# # duplicateRowsDF = estaciones1.Estacion[estaciones1.duplicated(['Estacion'])==True]
# duplicateRowsDF = estaciones.Estacion[estaciones.duplicated(['CodigoEst'])==True]

# # duplicateRowsDF = estaciones1.Estacion[estaciones1.duplicated(['Altitud','Xutm','Yutm'])==True] 
# print("Duplicate Rows except first occurrence based on geographic coordinates are :")
# print(duplicateRowsDF)

In [21]:
estaciones.drop_duplicates(subset='CodigoEst') # comprobar que no hay duplicados

,Estacion,CodigoEst,Tipo,Xutm,Yutm,Altura,Provincia,Umbral_20_40_km_costa
0,MIRONES,1104,INMman,442832.0,4793841.0,200.0,CN,2.0
1,RANON (AEROPUERTO DE ASTURIAS),1212E,INMman,255062.0,4828087.0,127.0,AS,1.0
2,GENESTOSO,1259,INMman,223429.0,4773596.0,1180.0,AS,3.0
3,A CORUNYA (ESTACION COMPLETA),1387,INMman,59149.0,4791283.0,58.0,AC,1.0
4,SANTIAGO COMPOSTELA (LABACOLLA,1428,INMman,55186.0,4739467.0,364.0,AC,2.0
5,VIGO (PEINADOR),1495,INMman,33443.0,4665653.0,255.0,PO,1.0
6,PONFERRADA,1549,INMman,204473.0,4718756.0,534.0,LE,3.0
7,NAVALENO,2080,INMman,499608.0,4631816.0,1105.0,SO,3.0
8,RIAZA,2125,INMman,459823.0,4569785.0,1180.0,SG,3.0
9,PANTANO DE REQUEJADA,2232,INMman,375123.0,4751856.0,1024.0,PA,3.0


## Merge DB

In [22]:
precipitacion.tail(3)

,CodigoEst,Fecha,Precipitacion,ValidPP
11109837,3010,1997-03-01,0.0,S
11109838,3010,1997-02-01,0.0,S
11109839,3010,1997-01-01,15.0,S


In [23]:
len(precipitacion.CodigoEst.unique())

905

In [24]:
len(estaciones.CodigoEst.unique())

943

In [77]:
estaciones.tail(3)

,Estacion,CodigoEst,Tipo,Xutm,Yutm,Altura,Provincia,Umbral_20_40_km_costa
940,ALIAGA (COMARCAL),9556U,INMman,693955.0,4504500.0,1115.0,TE,3.0
941,ALIAGA-SANTA BÁRBARA,9557B,INMman,692954.0,4505280.0,1105.0,TE,3.0
942,VALDERROBRES (COMARCAL),9935A,INMman,765138.0,4529462.0,482.0,TE,3.0


In [78]:
estaciones.Provincia.unique()

array(['CN', 'AS', 'AC', 'PO', 'LE', 'SO', 'SG', 'PA', 'BU', 'M', 'SL',
       'CU', 'TE', 'GU', 'TO', 'CA', 'CC', 'AB', 'BZ', 'HU', 'MU', 'JA',
       'CO', 'GR', 'CZ', 'A', 'V', 'C', 'TA'], dtype=object)

In [27]:
# une datos de las estaciones a la DB de precipitación
dataPP = pd.merge(precipitacion,
                estaciones[['Estacion','CodigoEst','Tipo','Xutm','Yutm','Altura','Provincia','Umbral_20_40_km_costa']],
                on='CodigoEst', how='left')

In [28]:
dataPP.describe(include='all')

,CodigoEst,Fecha,Precipitacion,ValidPP,Estacion,Tipo,Xutm,Yutm,Altura,Provincia,Umbral_20_40_km_costa
count,11109840,11109840,1.110984e+07,11109840,11023530,11023530,1.102353e+07,1.102353e+07,1.102353e+07,11023530,1.102353e+07
unique,905,41777,NaN,3,889,4,NaN,NaN,NaN,29,NaN
top,9981A,2003-04-27,NaN,S,"TORTOSA, OBSER. DEL EBRO",INMman,NaN,NaN,NaN,V,NaN
freq,36012,568,NaN,10589726,36012,10862482,NaN,NaN,NaN,2707979,NaN
mean,NaN,NaN,-2.719756e+00,NaN,NaN,NaN,6.525144e+05,4.357964e+06,5.465995e+02,NaN,2.265195e+00
std,NaN,NaN,2.097545e+01,NaN,NaN,NaN,1.080324e+05,1.101267e+05,4.191672e+02,NaN,8.566887e-01
min,NaN,NaN,-9.990000e+01,NaN,NaN,NaN,3.344300e+04,4.075366e+06,1.000000e+00,NaN,1.000000e+00
25%,NaN,NaN,0.000000e+00,NaN,NaN,NaN,6.127520e+05,4.277626e+06,1.270000e+02,NaN,1.000000e+00
50%,NaN,NaN,0.000000e+00,NaN,NaN,NaN,6.808070e+05,4.366168e+06,5.500000e+02,NaN,3.000000e+00
75%,NaN,NaN,0.000000e+00,NaN,NaN,NaN,7.198980e+05,4.427472e+06,8.880000e+02,NaN,3.000000e+00


In [29]:
dataPP.Provincia.unique()

array(['A', 'V', nan, 'AB', 'CU', 'C', 'TE', 'MU', 'CO', 'JA', 'M', 'CZ',
       'GR', 'TA', 'BU', 'CA', 'LE', 'SL', 'AC', 'PO', 'CN', 'AS', 'SO',
       'SG', 'PA', 'GU', 'TO', 'CC', 'BZ', 'HU'], dtype=object)

In [30]:
len(dataPP.CodigoEst.unique())

905

In [31]:
len(dataPP.Estacion.unique())

890

In [32]:
# une las temperaturas máx y mín a la DB precipitaciones

from functools import reduce

data_raw = reduce(lambda x,y: pd.merge(x,y, on=['CodigoEst','Fecha'], how='left'), [dataPP, t_max, t_min])

In [33]:
data_raw.describe(include='all')

,CodigoEst,Fecha,Precipitacion,ValidPP,Estacion,Tipo,Xutm,Yutm,Altura,Provincia,Umbral_20_40_km_costa,Tmaxima,ValidTmax,Tminima,ValidTmin
count,11109840,11109840,1.110984e+07,11109840,11023530,11023530,1.102353e+07,1.102353e+07,1.102353e+07,11023530,1.102353e+07,3.931106e+06,3931106,3.929750e+06,3929750
unique,905,41777,NaN,3,889,4,NaN,NaN,NaN,29,NaN,NaN,2,NaN,2
top,9981A,2003-04-27,NaN,S,"TORTOSA, OBSER. DEL EBRO",INMman,NaN,NaN,NaN,V,NaN,NaN,S,NaN,S
freq,36012,568,NaN,10589726,36012,10862482,NaN,NaN,NaN,2707979,NaN,NaN,3729738,NaN,3737798
mean,NaN,NaN,-2.719756e+00,NaN,NaN,NaN,6.525144e+05,4.357964e+06,5.465995e+02,NaN,2.265195e+00,2.051037e+01,NaN,9.608066e+00,NaN
std,NaN,NaN,2.097545e+01,NaN,NaN,NaN,1.080324e+05,1.101267e+05,4.191672e+02,NaN,8.566887e-01,8.698109e+00,NaN,6.936874e+00,NaN
min,NaN,NaN,-9.990000e+01,NaN,NaN,NaN,3.344300e+04,4.075366e+06,1.000000e+00,NaN,1.000000e+00,-9.900000e+01,NaN,-9.900000e+01,NaN
25%,NaN,NaN,0.000000e+00,NaN,NaN,NaN,6.127520e+05,4.277626e+06,1.270000e+02,NaN,1.000000e+00,1.500000e+01,NaN,4.500000e+00,NaN
50%,NaN,NaN,0.000000e+00,NaN,NaN,NaN,6.808070e+05,4.366168e+06,5.500000e+02,NaN,3.000000e+00,2.100000e+01,NaN,9.600000e+00,NaN
75%,NaN,NaN,0.000000e+00,NaN,NaN,NaN,7.198980e+05,4.427472e+06,8.880000e+02,NaN,3.000000e+00,2.700000e+01,NaN,1.500000e+01,NaN


## Limpieza de datos

### Índices no válidos

### Días sin precipitación

In [35]:
# no hay NaN en las precipitaciones (la londitud después de aplicar el dropna es la misma)
data_halfraw = data_raw.dropna(subset=['Precipitacion','ValidPP']) # elimina las filas que no tienen datos de precipitacion
data_halfraw.describe()

,Precipitacion,Xutm,Yutm,Altura,Umbral_20_40_km_costa,Tmaxima,Tminima
count,1.110984e+07,1.102353e+07,1.102353e+07,1.102353e+07,1.102353e+07,3.931106e+06,3.929750e+06
mean,-2.719756e+00,6.525144e+05,4.357964e+06,5.465995e+02,2.265195e+00,2.051037e+01,9.608066e+00
std,2.097545e+01,1.080324e+05,1.101267e+05,4.191672e+02,8.566887e-01,8.698109e+00,6.936874e+00
min,-9.990000e+01,3.344300e+04,4.075366e+06,1.000000e+00,1.000000e+00,-9.900000e+01,-9.900000e+01
25%,0.000000e+00,6.127520e+05,4.277626e+06,1.270000e+02,1.000000e+00,1.500000e+01,4.500000e+00
50%,0.000000e+00,6.808070e+05,4.366168e+06,5.500000e+02,3.000000e+00,2.100000e+01,9.600000e+00
75%,0.000000e+00,7.198980e+05,4.427472e+06,8.880000e+02,3.000000e+00,2.700000e+01,1.500000e+01
max,8.780000e+02,8.150770e+05,4.828087e+06,1.955000e+03,3.000000e+00,5.270000e+01,3.810000e+01


### Datos de estaciones desconocidas

In [36]:
data_halfraw.isnull().sum()    # no NaN data in Precipitacion, only in Temperatures

CodigoEst                      0
Fecha                          0
Precipitacion                  0
ValidPP                        0
Estacion                   86310
Tipo                       86310
Xutm                       86310
Yutm                       86310
Altura                     86310
Provincia                  86310
Umbral_20_40_km_costa      86310
Tmaxima                  7178734
ValidTmax                7178734
Tminima                  7180090
ValidTmin                7180090
dtype: int64

In [45]:
# estaciones desconocidas con pp
data_halfraw.CodigoEst[data_halfraw.Estacion.isnull()==True].unique()

array(['8041A', '8041C', '8040C', '8411C', '8041', '8008A', '8325C'],
      dtype=object)

In [48]:
data_halfraw = data_halfraw[~data_halfraw.CodigoEst.
                               isin(['8041A', '8041C', '8040C', '8411C', '8041', '8008A', '8325C'])
                              ]

In [49]:
len(data_halfraw.CodigoEst.unique())

898

### Datos de precipitación NO válidos

In [50]:
data_halfraw.ValidPP.unique()           #  ValidPP, a parte de S(í) y N(o), hay un campo A(cumulado)

array(['S', 'N', 'A'], dtype=object)

In [61]:
# Input
ValidPP = ['N', 'A']          # casos con validez diferente a 'S'
value = [-99.9, -0.4,-0.3, 0] # valores concretos con precipitación menor que 0

print ('--- Validez de la precipitación ---')
print("\n")

for each1 in ValidPP:
    lentot = len(data_halfraw[(data_halfraw.ValidPP == each1)])
    lenpos = len(data_halfraw[(data_halfraw.ValidPP == each1) & (data_halfraw.Precipitacion > 0)])
    print ('Hay '+str(lentot)+' casos con validez '+ each1+' , de los cuales '+str(lenpos) +' tienen precipitacion > 0')
    for each2 in value:
        lenneg = len(data_halfraw[(data_halfraw.ValidPP == each1) & (data_halfraw.Precipitacion == each2)])
        print('    Cuando la validez es '+ each1 +' y la cantidad es ' + str(each2)+' mm, hay '+ str(lenneg) +' casos' )
    print("\n")

--- Validez de la precipitación ---


Hay 43555 casos con validez N , de los cuales 0 tienen precipitacion > 0
    Cuando la validez es N y la cantidad es -99.9 mm, hay 0 casos
    Cuando la validez es N y la cantidad es -0.4 mm, hay 0 casos
    Cuando la validez es N y la cantidad es -0.3 mm, hay 43555 casos
    Cuando la validez es N y la cantidad es 0 mm, hay 0 casos


Hay 6 casos con validez A , de los cuales 0 tienen precipitacion > 0
    Cuando la validez es A y la cantidad es -99.9 mm, hay 0 casos
    Cuando la validez es A y la cantidad es -0.4 mm, hay 0 casos
    Cuando la validez es A y la cantidad es -0.3 mm, hay 6 casos
    Cuando la validez es A y la cantidad es 0 mm, hay 0 casos




In [60]:
# elimina validez N (linea 1) y elimina validez A solo en el caso que pp = -99.9 mm 

data_halfraw = data_halfraw.drop(data_halfraw[(data_halfraw['ValidPP'] == 'N') & (data_halfraw.Precipitacion != (-0.3)) |    
                                (data_halfraw['ValidPP'] == 'A') & (data_halfraw.Precipitacion != (-0.3))].index)

560040

In [ ]:
len(data_raw)-len(data_halfraw)

### Conversión lluvia insignificante (-0.3 a 0)

In [67]:
data_halfraw.Precipitacion[data_halfraw.Precipitacion == (-0.3)] = 0

C:\Users\ceam\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [68]:
len(data_halfraw[data_halfraw.Precipitacion == 0])

8857075

In [69]:
len(data_halfraw[(data_halfraw.Precipitacion == (-0.3))])

0

### Datos de temperatura NO válidos

In [70]:
# elimina SOLO las temperaturas cuando el dato no es válido, deja el resto de la fila

data_halfraw.Tmaxima[data_halfraw['ValidTmax'] == 'N'] = np.nan
data_halfraw.Tminima[data_halfraw['ValidTmin'] == 'N'] = np.nan

C:\Users\ceam\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ceam\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


### Datos de fecha fuera de rango

In [71]:
# con la anterior limpieza (por ValidPP), ha desaparecido también los errores de fecha (out of range)

len(data_halfraw[data_halfraw.Fecha>'2019-06-30'])                     # datos en 2019: 1805, todos de enero
len(data_halfraw.CodigoEst[data_halfraw.Fecha>'2018-12-31'].unique())  # número de estaciones con datos en 2019: 60

len(data_halfraw[(data_halfraw.Fecha<'1900-01-01') & (data_halfraw.Fecha>'2019-06-30')]) # no data out of range

0

In [72]:
# número de estaciones cuyos datos fueron todos eliminados por validez de precipitación

a = data_halfraw.CodigoEst.unique() # estaciones con datos válidos
b = data_raw.CodigoEst.unique()     # estaciones con datos

main_list = np.setdiff1d(b,a)
main_list                            # lista de estaciones sin datos válidos (8096 = Cuenca)

#len(b)-len(a)                     #  número estaciones sin datos válidos

array(['8008A', '8040C', '8041', '8041A', '8041C', '8325C', '8411C'],
      dtype=object)

### Formatos uniformes

In [73]:
# longitud de cada uno de los items de una Serie (columna). para saber si uniforme, usar mean() o mejor, describe()

data_halfraw.Fecha.str.len().describe()

count    10549800.0
mean           10.0
std             0.0
min            10.0
25%            10.0
50%            10.0
75%            10.0
max            10.0
Name: Fecha, dtype: float64

In [74]:
data_halfraw.Xutm.astype('str').str.len().describe()    # num de dígitos en Xutm

count    1.054980e+07
mean     7.993036e+00
std      8.316130e-02
min      7.000000e+00
25%      8.000000e+00
50%      8.000000e+00
75%      8.000000e+00
max      8.000000e+00
Name: Xutm, dtype: float64

In [75]:
# longitud de Xutm = 7 en estacions de galicia (A Corunya, Vigo)
data_halfraw.Provincia[data_halfraw.Xutm.astype('str').str.len()==7].unique()

array(['AC', 'PO'], dtype=object)

In [76]:
data_halfraw.Yutm.astype('str').str.len().describe()    # num de dígitos en Yutm

count    10549800.0
mean            9.0
std             0.0
min             9.0
25%             9.0
50%             9.0
75%             9.0
max             9.0
Name: Yutm, dtype: float64

In [76]:
data_halfraw.tail()

,CodigoEst,Fecha,Precipitacion,ValidPP,Estacion,Tipo,Xutm,Yutm,Altura,Provincia,Umbral_20_40_km_costa,Tmaxima,ValidTmax,Tminima,ValidTmin
3562254,8073,2018-11-26,0.0,S,XERESA,INMman,740848.0,4321455.0,25.0,V,1.0,NaN,NaN,NaN,NaN
3562239,8073,2018-11-27,0.0,S,XERESA,INMman,740848.0,4321455.0,25.0,V,1.0,NaN,NaN,NaN,NaN
3562224,8073,2018-11-28,0.0,S,XERESA,INMman,740848.0,4321455.0,25.0,V,1.0,NaN,NaN,NaN,NaN
3562210,8073,2018-11-29,0.0,S,XERESA,INMman,740848.0,4321455.0,25.0,V,1.0,NaN,NaN,NaN,NaN
3562196,8073,2018-11-30,0.0,S,XERESA,INMman,740848.0,4321455.0,25.0,V,1.0,NaN,NaN,NaN,NaN


In [77]:
data_halfraw.dtypes                                     # formato de las columnas de mi DB

CodigoEst                 object
Fecha                     object
Precipitacion            float64
ValidPP                   object
Estacion                  object
Tipo                      object
Xutm                     float64
Yutm                     float64
Altura                   float64
Provincia                 object
Umbral_20_40_km_costa    float64
Tmaxima                  float64
ValidTmax                 object
Tminima                  float64
ValidTmin                 object
dtype: object

### Estaciones fuera de rango geogràfico

In [78]:
len(data_halfraw.CodigoEst.unique())

898

In [79]:
# Eliminar datos de estaciones que no sean C, V, A, MU, AB, CU, TE

data_halfraw = data_halfraw[data_halfraw.Provincia.isin(['CU', 'AB','MU','TE','C','V','A'])]

In [80]:
len(data_halfraw.CodigoEst.unique())

862

### Ordenar

In [81]:
### La jerarquía es: 1. Provincia, 2. Nombre de estación, 3. Código de la Est, 4. Fecha

data_halfraw.sort_values(by=['Provincia','Estacion','CodigoEst','Fecha'], inplace=True)

In [83]:
data_halfraw.tail()

,CodigoEst,Fecha,Precipitacion,ValidPP,Estacion,Tipo,Xutm,Yutm,Altura,Provincia,Umbral_20_40_km_costa,Tmaxima,ValidTmax,Tminima,ValidTmin
10444025,8073,2018-11-26,0.0,S,XERESA,INMman,740848.0,4321455.0,25.0,V,1.0,NaN,NaN,NaN,NaN
10444010,8073,2018-11-27,0.0,S,XERESA,INMman,740848.0,4321455.0,25.0,V,1.0,NaN,NaN,NaN,NaN
10443995,8073,2018-11-28,0.0,S,XERESA,INMman,740848.0,4321455.0,25.0,V,1.0,NaN,NaN,NaN,NaN
10443981,8073,2018-11-29,0.0,S,XERESA,INMman,740848.0,4321455.0,25.0,V,1.0,NaN,NaN,NaN,NaN
10443967,8073,2018-11-30,0.0,S,XERESA,INMman,740848.0,4321455.0,25.0,V,1.0,NaN,NaN,NaN,NaN


### Dividir fecha en 3 columnas

In [84]:
#adding Year Month Day columns

data_halfraw['Year']=[d.split('-')[0] for d in data_halfraw.Fecha]
data_halfraw['Month']=[d.split('-')[1] for d in data_halfraw.Fecha]
data_halfraw['Day']=[d.split('-')[2] for d in data_halfraw.Fecha]

data_halfraw.tail(5)

,CodigoEst,Fecha,Precipitacion,ValidPP,Estacion,Tipo,Xutm,Yutm,Altura,Provincia,Umbral_20_40_km_costa,Tmaxima,ValidTmax,Tminima,ValidTmin,Year,Month,Day
10444025,8073,2018-11-26,0.0,S,XERESA,INMman,740848.0,4321455.0,25.0,V,1.0,NaN,NaN,NaN,NaN,2018,11,26
10444010,8073,2018-11-27,0.0,S,XERESA,INMman,740848.0,4321455.0,25.0,V,1.0,NaN,NaN,NaN,NaN,2018,11,27
10443995,8073,2018-11-28,0.0,S,XERESA,INMman,740848.0,4321455.0,25.0,V,1.0,NaN,NaN,NaN,NaN,2018,11,28
10443981,8073,2018-11-29,0.0,S,XERESA,INMman,740848.0,4321455.0,25.0,V,1.0,NaN,NaN,NaN,NaN,2018,11,29
10443967,8073,2018-11-30,0.0,S,XERESA,INMman,740848.0,4321455.0,25.0,V,1.0,NaN,NaN,NaN,NaN,2018,11,30


### Añadir distancia estación-costa

In [85]:
path4 = r'C:/CicloHidrico/Datos'      # directorio de los datos
os.chdir(path4)

In [86]:
# estaciones con datos en la CHJ
est_CHJ = data_halfraw.drop_duplicates(['CodigoEst'])

In [87]:
# en csv, para cálculo de dist_costa (Hassane)
est_CHJ.to_csv('est_CHJ.csv')

In [88]:
est = pd.read_excel("EstacionesCHJ09_2019.xlsx",index_col=0)  # con las dist_costa

In [89]:
est.CodigoEst = est.CodigoEst.astype('str')

In [90]:
# merge distance to coast to the last column
data_halfraw = data_halfraw.merge(est[['CodigoEst','dist_costa']],on='CodigoEst',how='left')

In [91]:
data_halfraw.dist_costa.count()

9814483

# To Excel

In [92]:
meteodata = data_halfraw
del data_halfraw

In [116]:
# reset index per a que estiga ordenat de 0 a X
# meteodata.reset_index(drop=True, inplace=True)

In [93]:
meteodata.to_csv('meteodata.csv')

In [ ]:
meteodata[(meteodata.Precipitacion > 400)]

In [96]:
meteodata.describe(include='all')

,CodigoEst,Fecha,Precipitacion,ValidPP,Estacion,Tipo,Xutm,Yutm,Altura,Provincia,Umbral_20_40_km_costa,Tmaxima,ValidTmax,Tminima,ValidTmin,Year,Month,Day,dist_costa
count,9814483,9814483,9.814483e+06,9814483,9814483,9814483,9.814483e+06,9.814483e+06,9.814483e+06,9814483,9.814483e+06,3.554167e+06,3694386,3.562843e+06,3692830,9814483,9814483,9814483,9.814483e+06
unique,862,39118,NaN,3,853,4,NaN,NaN,NaN,7,NaN,NaN,2,NaN,2,108,12,31,NaN
top,7102,2003-04-10,NaN,S,ONTUR GRUPO ESCOLAR,INMman,NaN,NaN,NaN,V,NaN,NaN,S,NaN,S,2003,05,11,NaN
freq,34009,531,NaN,9770922,34009,9657591,NaN,NaN,NaN,2594822,NaN,NaN,3554167,NaN,3562843,187472,841875,322639,NaN
mean,NaN,NaN,1.328469e+00,NaN,NaN,NaN,6.737372e+05,4.347105e+06,5.290456e+02,NaN,2.233426e+00,2.137944e+01,NaN,1.004954e+01,NaN,NaN,NaN,NaN,5.974007e+01
std,NaN,NaN,6.246974e+00,NaN,NaN,NaN,6.161864e+04,9.142099e+04,4.147087e+02,NaN,8.588786e-01,7.809340e+00,NaN,6.800974e+00,NaN,NaN,NaN,NaN,5.360077e+01
min,NaN,NaN,0.000000e+00,NaN,NaN,NaN,5.370000e+05,4.141490e+06,1.000000e+00,NaN,1.000000e+00,-1.500000e+01,NaN,-2.700000e+01,NaN,NaN,NaN,NaN,-1.000000e+01
25%,NaN,NaN,0.000000e+00,NaN,NaN,NaN,6.284940e+05,4.275126e+06,1.000000e+02,NaN,1.000000e+00,1.550000e+01,NaN,5.000000e+00,NaN,NaN,NaN,NaN,1.552000e+01
50%,NaN,NaN,0.000000e+00,NaN,NaN,NaN,6.865970e+05,4.355969e+06,5.150000e+02,NaN,3.000000e+00,2.100000e+01,NaN,1.000000e+01,NaN,NaN,NaN,NaN,4.118000e+01
75%,NaN,NaN,0.000000e+00,NaN,NaN,NaN,7.205340e+05,4.422256e+06,8.680000e+02,NaN,3.000000e+00,2.770000e+01,NaN,1.500000e+01,NaN,NaN,NaN,NaN,9.596000e+01
